In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

from os import path

In [ ]:
# CONECTAR A BIGQUERY
base_path = '/content/drive/MyDrive/DMEyF/2025/'
dataset_path = base_path + 'datos/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_01.csv'

ganancia_acierto = 780000
costo_estimulo = 20000

data = pd.read_csv(path.join(dataset_path,dataset_file))

In [ ]:

mes_train = 202102
mes_score = 202104



In [ ]:
train_data = data[data['foto_mes'] == mes_train]
score_data = data[data['foto_mes'] == mes_score]

train_null_percentage = train_data.isnull().mean() * 100
score_null_percentage = score_data.isnull().mean() * 100

comparison_df = pd.DataFrame({'Train Null Percentage': train_null_percentage, 'Score Null Percentage': score_null_percentage})
comparison_df['diff'] = (comparison_df['Score Null Percentage'] - comparison_df['Train Null Percentage']).abs()

comparison_df_sorted = comparison_df.sort_values('diff', ascending=False)

comparison_df_sorted


In [ ]:
train_zero_percentage = (train_data == 0).mean() * 100
score_zero_percentage = (score_data == 0).mean() * 100

comparison_df_zero = pd.DataFrame({'Train Zero Percentage': train_zero_percentage, 'Score Zero Percentage': score_zero_percentage})

comparison_df_zero['diff_zero_percentage'] = (comparison_df_zero['Score Zero Percentage'] - comparison_df_zero['Train Zero Percentage']).abs()
diff_zero_percentage_sorted = comparison_df_zero.sort_values('diff_zero_percentage',ascending=False)
diff_zero_percentage_sorted


In [ ]:

def psi(expected, actual, buckets=10):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
    bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
    breakpoints = sorted(list(set(bin_edges2)))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null



buckets=10
column = 'mprestamos_personales'
expected = train_data[column]
actual = score_data[column]

def psi_formula(expected_prop, actual_prop):
    result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
    return result

expected_not_null = expected.dropna()
actual_not_null = actual.dropna()

bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
breakpoints = sorted(list(set(bin_edges2)))

expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

expected_prop = expected_counts / len(expected_not_null)
actual_prop = actual_counts / len(actual_not_null)

psi_not_null = psi_formula(expected_prop, actual_prop).sum()

psi_null = 0

psi_value = psi(expected, actual)



In [ ]:
buckets=10
column = 'mprestamos_personales'
expected = train_data[column]
actual = score_data[column]

def psi_formula(expected_prop, actual_prop):
    result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
    return result

expected_not_null = expected.dropna()
actual_not_null = actual.dropna()

bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
breakpoints = sorted(list(set(bin_edges2)))

expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

expected_prop = expected_counts / len(expected_not_null)
actual_prop = actual_counts / len(actual_not_null)

psi_not_null = psi_formula(expected_prop, actual_prop).sum()

psi_null = 0

psi_value = psi(expected, actual)



In [ ]:
psi_results = []
for column in train_data.columns:
  if column not in ['foto_mes', 'clase_ternaria', 'ctarjeta_master_descuentos','mtarjeta_master_descuentos','mtarjeta_visa_descuentos','ctarjeta_visa_descuentos','ccajeros_propios_descuentos','mcajeros_propios_descuentos']:
    print(column)
    train_variable = train_data[column]
    score_variable = score_data[column]
    psi_value = psi(train_variable, score_variable)
    psi_results.append({'feature': column, 'psi': psi_value})

psi_df = pd.DataFrame(psi_results)
psi_df = psi_df.sort_values('psi', ascending=False)
psi_df